# Generate Samples
This notebook genertes 100 samples of solar minimum and solar maximum, and saves in the `plots/samples` directory in the main repository. Fixed seeds are used for reproducability. 

In [ ]:
import os
import sys

sys.path.append('..')
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import torch
import numpy as np
import random
import pylab as plt
import tqdm

from coronal_diffusion import dataset
from coronal_diffusion import visualization_tools as vt
from coronal_diffusion import sampler

import config

In [ ]:
def set_seeds(seed):
    torch.use_deterministic_algorithms(True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
os.chdir('..')
print('Current directory is:', os.getcwd())

In [ ]:
plot_dir = 'plots/samples/'
weights_file = "checkpoints/experiment14-refactor-alldata_2.pth"
nitems = 100
tasks = [(0, 'min', config.seed_helper_min),
         (1, 'max', config.seed_helper_max)]

!rm -rf $plot_dir
!mkdir -p $plot_dir

In [ ]:
for radio_flux, tag, seed_helper in tasks:
    for i in tqdm.tqdm(list(range(1, nitems + 1)), desc=f'Solar {tag}'):
        set_seeds(i)
        G, H = sampler.sample(weights_file=weights_file, radio_flux=radio_flux, method='ddim', seed_helper=seed_helper)
        sh = vt.SHVisualizer(G, H)
        fig = sh.visualize_field_lines(r=1.1, grid_density=20)
        fig.write_image(f'{plot_dir}/{tag}_seed{i}.png', width=800, height=600)
        fig.data = []